## Model Exploration (ML50-2023)

### __Importing libraries__

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from scipy.stats import randint


# Data imports
test = pd.read_csv('Data/test_cleaned.csv', index_col=0)
train = pd.read_csv('Data/train_cleaned.csv', index_col=0)

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import RandomizedSearchCV

# Model imports
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# Metrics imports
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
def test_model(model, X_train, y_train, X_test, y_test, just_score=False): 
    fit = model.fit(X_train, y_train)
    y_pred = fit.predict(X_test)
    if just_score:
        return fit, f1_score(y_test, y_pred)
    
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('Precision: ', precision_score(y_test, y_pred))
    print('Recall: ', recall_score(y_test, y_pred))
    print('F1: ', f1_score(y_test, y_pred))
    print('Confusion matrix: \n', confusion_matrix(y_test, y_pred))
    print('Classification report: \n', classification_report(y_test, y_pred))
    return fit

### __Feature Selection__

#### __Using a random forest__
__Modifications:__

<br>

__Relevant Information:__


In [ ]:
def get_train_val(data, test_size=0.2, target='b_target', scaler=MinMaxScaler(), sampler=RandomOverSampler()):
    y = data[target]
    X = data.drop(columns=['b_target', 'c_target'])
    numeric_cols = [col for col in X.columns if col.startswith('n_')]

    if scaler:
        X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
        
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42)

    if sampler:
        X_train, y_train = sampler.fit_resample(X_train, y_train)

    return X_train, X_val, y_train, y_val

In [ ]:
def perform_random_search(data, model, params_dict, sampler=RandomOverSampler(), scaler=MinMaxScaler(), target='b_target'):
   # Create a pipeline
   pipeline = ImbPipeline([
       ('sampler', sampler),
       ('scaler', scaler),
       ('model', model)
   ])

   y = data[target]
   X = data.drop(columns=['b_target', 'c_target'], errors='ignore')

   # Perform randomized search
   random_search = RandomizedSearchCV(pipeline, params_dict, cv=5, n_jobs=-1, random_state=42, verbose=1, scoring='f1')
   random_search.fit(X, y)

   # Print the best parameters
   print("Best parameters: ", random_search.best_params_)

   # Return the best estimator
   return random_search.best_estimator_

In [27]:
params_dict = {
  'model__n_estimators': [val*20 for val in range(1, 50)],
  'model__max_depth': [val*2 for val in range(1, 30)],
  'model__min_samples_split': randint(2, 10),
  'model__min_samples_leaf': randint(1, 10),
  'model__max_features': ['sqrt', 'log2'],
  'model__bootstrap': [True, False],
  'model__criterion': ['gini', 'entropy'],
  'model__class_weight': ['balanced', 'balanced_subsample', None]
}

best_estimator = perform_random_search(train, RandomForestClassifier(), params_dict)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\mateus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mateus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mateus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-p

Best parameters:  {'model__bootstrap': True, 'model__class_weight': 'balanced_subsample', 'model__criterion': 'gini', 'model__max_depth': 4, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 6, 'model__min_samples_split': 3, 'model__n_estimators': 420}


In [ ]:
# Fit a random forest classifier
rf_classifier = RandomForestClassifier(bootstrap=True, class_weight='balanced_subsample', criterion='gini',max_depth=4, max_features='sqrt', min_samples_leaf=6, min_samples_split=3, n_estimators=420)

X_train, X_val, y_train, y_val = get_train_val(train, scaler=MinMaxScaler(), sampler=RandomOverSampler())